 # <center> Identifying Turnover in Mortgage Prepayments <center>
## <center> US Bank, IMA Math to Industry Boot Camp 2022 <center>
### <center> Caroline Bang, Katheryn Beck, Zanbing Dai, Leo Digiosia, Arpan Pal <center>

U.S. agency 1 residential mortgage-backed securities (MBS) are the largest and most liquid securitized asset class in the world. Banks, insurance companies, and money managers invest in MBS because they provide an attractive yield relative to U.S. Treasury securities with comparable credit risk. However, unlike most fixed income securities, which have specified contractual coupon and principal payments, the timing and amount of MBS cash flows is uncertain. This is because MBS are pools of individual mortgages on which the borrower has the right to prepay the loan at any given time during the life of the loan. Prepayment risk, which impacts the yield and interest rate risk of an MBS comes from five sources:

 - Rate/term refinance which occur when borrowers lower interest payments or shorten the term of the current mortgage.
 - Cash-out refinance which involve extracting equity from a home.
 - Involuntary buyouts, which in the case of agency MBS result in an early return of principal. However, the timing is contingent on the GSEs or GNMA loan servicers.
 - Curtailments, which are partial prepayment or full payoff before maturity.
 - Turnover, which is caused by geographic migration and home upgrades. Turnover creates a baseline level of prepayments that are highly seasonal.

The COVID-19 crisis and response by the Federal Reserve resulted in a low-rate environment that elevated both levels of refinance and buyout activity. Since the initiation of quantitative tightening, sustained inflation, and rising interest rates, refinance activity has slowed considerably. At present, a major question affecting the risk profile of MBS is to what extent prepayment activity will decrease. With diminishing refinance incentives, the major sources of prepayment are now turnover, curtailment, and buyouts. As a baseline level of prepayment activity, understanding turnover is important to evaluating the risks of mortgage-backed securities. The goal of this project is to use loan-level mortgage data and macroeconomic data to quantify turnover prepayment speed.

---------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------

# (1) Using incentives to distinguish refinancing from turnover

Turnover and refinancing are the most common reasons for mortgage prepayment. Associated to a pool of mortgages are two quantities:

 - CPR, The *Conditional prepayment rate*. This is a percentage describing what proportion of the loan pool is being paid ahead of schedule per year. A high CPR indicates that many mortgages in a given pool are experiencing prepayment
 - WAC, the *Weighted Average Coupon* associated to the pool describes the average mortgage rate over all loans.
 
Given a mortgage pool with its associated WAC, we can compute its **refinance incentive**: 

$$\text{incentive} = \text{WAC} - \text{Mortgage},$$

where the Mortgage term describes the mortgage rate that borrowers in the pool could obtain by refinancing their mortgage. Thus, when this difference is high and positive, we expect that borrowers will take the incentive, refinance their mortgage, and pay off their existing mortgage in full, contributing to an increase in CPR for that pool. When the incentive is negative, we assume that borrowers who pay their mortgage in full are not doing so due to refinancing, but instead, due to turnover.

# (2) Using HPI and Housing Credit Availability to predict turnover